In [7]:
tc = [
    ([[2, 3], [4, 3], [1, 1], [2, 1]], [2, 1, 1, 0]),
    ([[4, 11], [1, 12], [8, 3], [12, 7], [4, 2], [7, 11], [4, 8], [9, 6], [10, 11], [6, 10], [3, 5], [11, 1], [5, 3], [11, 9], [3, 8]], [4, 0, 1, 2])
]

In [26]:
def solution(edges):
    answer = []
    
    edges = [(src, dst) for src, dst in edges]
    
    # connected component

    
    def find_outgoing_edges(vertex):
        return [e for e in edges if e[0] == vertex]
    
    
    def find_incoming_edges(vertex):
        return [e for e in edges if e[1] == vertex]
    
    
    def has_outgoing_edge(vertex):
        for edge in edges:
            if edge[0] == vertex:
                return True
        return False
    
    
    def get_all_vertex_numbers():
        numbers = set()
        for src, dst in edges:
            numbers.add(src)
            numbers.add(dst)
        return list(numbers)
    
    
    def get_vertex_outdegrees():
        INF = int(1e9)
        vertex_numbers = get_all_vertex_numbers()
        vertex_outdegrees = [INF] * (max(vertex_numbers) + 1)
        
        for v in vertex_numbers:
            vertex_outdegrees[v] = len(find_outgoing_edges(v))
            
        return vertex_outdegrees
    
    
    def get_vertex_indegrees():
        INF = int(1e9)
        vertex_numbers = get_all_vertex_numbers()
        vertex_indegrees = [INF] * (max(vertex_numbers) + 1)
        
        for v in vertex_numbers:
            vertex_indegrees[v] = len(find_incoming_edges(v))
            
        return vertex_indegrees
    
    
    def get_cc(vertex):
        # Find a connected component about 'vertex'
        vertex_nums = get_all_vertex_numbers()
        visited = [False] * (max(vertex_nums) + 1)
        
        def dfs(vertex):
            visited[vertex] = True
            
            for src, dst in edges:
                if src == vertex:
                    if not visited[dst]:
                        dfs(dst)
        
        dfs(vertex)
        
        cc = []
        for i in range(1, len(visited)):
            if visited[i]:
                cc.append(i)
        
        return cc
    
    
    def get_donut_edges_or_none(vertex):
        if not has_outgoing_edge(vertex):
            return None
        
        cc = get_cc(vertex)
        
        cursor = vertex
        donut_edges = []
        check = True
        for i in range(len(cc)):
            outgoing_edges = find_outgoing_edges(cursor)
            if len(outgoing_edges) == 1:
                cursor = outgoing_edges[0][1]
                donut_edges.append(outgoing_edges[0])
            else:
                check = False
                break
        
        if not check:
            return None
        
        if cursor == vertex:
            print("donut cc: ", cc)
            return donut_edges
        else:
            return None
    
    
    def get_stick_edges_or_none(vertex):
        pass
    
    
    def get_eight_edges_or_none(vertex):
        if not has_outgoing_edge(vertex):
            return None
        
        cc = get_cc(vertex)
        eight_edges = set()
        for v in cc:
            o = find_outgoing_edges(v)
            i = find_incoming_edges(v)
            eight_edges.update(o, i)
        
        if not ( (len(cc) % 2 == 1) and (len(cc) + 1 == len(eight_edges)) ):
            return None
     
        print("eight cc: ", cc)
        return list(eight_edges)
    
    
    
    
    
    original_outdegrees = get_vertex_outdegrees()
    original_indegrees = get_vertex_indegrees()
    original_vertex_numbers = get_all_vertex_numbers()
    visited = [False] * (max(original_vertex_numbers) + 1)
    donut_count = 0
    stick_count = 0
    eight_count = 0
    new_vertex = None
    
    for v in get_all_vertex_numbers():
        if original_outdegrees[v] >= 2 and original_indegrees[v] == 0:
            new_vertex = v
            # edges = [e for e in edges if e not in find_outgoing_edges(new_vertex)]
            break
    
    print(f"new_vertex: {new_vertex}")
    
    vertices = get_all_vertex_numbers()
    for v in vertices:
        donut_edges = get_donut_edges_or_none(v)
        if donut_edges is not None:
            print("donut 발견!")
            edges = [e for e in edges if e not in donut_edges]
            donut_count += 1
    
    
    vertices = get_all_vertex_numbers()
    for v in vertices:
        eight_edges = get_eight_edges_or_none(v)
        
        if eight_edges is not None:
            print("eight 발견!")
            edges = [e for e in edges if e not in eight_edges]
            eight_count += 1
   

    
            
    # Now, there are only stick graphs.
    remaining_vertices = get_all_vertex_numbers()
    print("remaining_vertices: ", remaining_vertices)
    indegrees = get_vertex_indegrees()
    stick_count = len([i for i in indegrees if i == 0])
    
    
    
    answer = [new_vertex, donut_count, stick_count, eight_count]
    
    
    
    # 색칠이 안 된 vertex가 바로 새로 생성한 vertex OR 2개 이상의 다른 CC에 포함되는 vertex를 
    
    
    
    return answer



def expect(sol, answer):
    if sol != answer:
        print(f"틀렸음! 정답: {sol}, 하지만 나의 답: {answer}")
    else:
        print("정답!")

# 생성한 정점의 번호, 도넛 모양 그래프의 수, 막대 모양 그래프의 수, 8자 모양 그래프의 수        
for t in tc:
    expect(t[1], solution(t[0]))


new_vertex: 2
donut cc:  [1]
donut 발견!
remaining_vertices:  [1, 2, 3, 4]
틀렸음! 정답: [2, 1, 1, 0], 하지만 나의 답: [2, 1, 2, 0]
new_vertex: 4
remaining_vertices:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
틀렸음! 정답: [4, 0, 1, 2], 하지만 나의 답: [4, 0, 1, 0]


In [50]:
from collections import deque


def solution(edges):
    answer = []
    
    edges = [(s, d) for s, d in edges]
    
    # 현재 그래프에 남아 있는 vertex queue
    q = deque()
    temp = set()
    for src, dst in edges:
        temp.add(src)
        temp.add(dst)
    
    for v in temp:
        q.append(v)
    
    max_vertex_number = max(q)
    
    def get_cc(vertex, graph_edges):
        # Find a connected component about 'vertex'
        
        visited = [False] * (max_vertex_number + 1)
        
        def dfs(vertex):
            visited[vertex] = True
            
            for src, dst in graph_edges:
                if src == vertex:
                    if not visited[dst]:
                        dfs(dst)
        
        dfs(vertex)
        
        cc = []
        for i in range(1, len(visited)):
            if visited[i]:
                cc.append(i)
        
        return cc
    
    
    def init_out_and_in_edges():
        out_edges = dict()
        in_edges = dict()
        
        # q 주의해야 한다.
        for v in q:
            out_edges[v] = list()
            in_edges[v] = list()
            
        for src, dst in edges:
            out_edges[src].append( (src, dst) )
            in_edges[dst].append( (src, dst) )
            
        return out_edges, in_edges
    
    # vertex 번호로 outgoing edges 찾기
    # vertex 번호로 incoming edges 찾기
    
    out_edges, in_edges = init_out_and_in_edges()
    
    
    # 새로 생성한 vertex
    # |outgoing edges| >= 2, |incoming edges| == 0
    
    new_vertex = None
    for v in q:
        if len(out_edges[v]) >=2 and len(in_edges[v]) == 0:
            new_vertex = v
    
    if new_vertex is None:
        raise Exception("unreachable")
    
    q.remove(new_vertex)
    
    #print("out_edges[new_vertex]: ", out_edges[new_vertex])
    
    # 새로 생성한 vertex를 제거함으로써, 그래프가 여러 부분들로 끊어진다.
    edges = [e for e in edges if e not in out_edges[new_vertex]]
    
    # renew the out/in edges 
    out_edges, in_edges = init_out_and_in_edges()
    
    # 도넛 모양
    # |v| = n, |e| = n
    # 어느 vertex에서 출발하든 한 바퀴 순회 가능
    def donut(vertex):
        #print("\tdonut vertex: ", vertex)
        cc = get_cc(vertex, edges)
        #print("\tdonut cc: ", cc)
        
        if len(cc) == 0:
            #print("\tdonut fail; cc length check.")
            return None
        
        for v in cc:
            l = len(out_edges[v])
            if l != 1:
                #print("\tdonut edges: ", edges)
                #print("\tdonut out_edges[v]: ", out_edges[v])
                #print(f"\tdonut fail; out edge of ({v}) length check.")
                return None
        
        cur = cc[0]
        for _ in range(len(cc)):
            _, dst = out_edges[cur][0]
            cur = dst
        
        if cur == cc[0]:
            return cc
        else:
            #print(f"\tdonut fail; loop check")
            return None    
        
    
    # 막대 모양
    # |v| = n, |e| = n - 1
    # 임의의 vertex에서 출발해서 순회한 개수 누적하기, 누적 개수 == n - 1으로 끝나면 막대기
    # vertex의 outgoing edge가 최대 1개
    
    def stick(vertex):
        #print("1stick vertex: ", vertex)
        #print("in_edges[vertex]: ", in_edges[vertex])
        if len(in_edges[vertex]) != 0:
            return None
        
        #print("edges: ", edges)
        cc = get_cc(vertex, edges)
        
        #print("2stick vertex: ", vertex)
        #print("stick cc: ", cc)
        count_v = len(cc)
        count_e = 0
        for v in cc:
            l = len(out_edges[v])
            if l >= 2:
                return None
            count_e += l
        
        if count_v - 1 != count_e:
            return None

        return cc
    
    
    # 8자 모양
    # |v| = 2n + 1 (홀수) , |e| = 2n + 2
    def eight(vertex):
        
        cc = get_cc(vertex, edges)
        
        count_v = len(cc)
        count_e = 0
        for v in cc:
            l = len(out_edges[v])
            count_e += l
        
        if count_v % 2 == 1 and (count_v + 1 == count_e):
            return cc
        else:
            return None
    
    donut_count = 0
    stick_count = 0
    eight_count = 0
    while q:
        v = q.popleft()
        
        cc = donut(v)
        if cc:
            donut_count += 1
            for i in cc:
                edges = [e for e in edges if e not in out_edges[i]]
                if i in q:
                    q.remove(i)
            continue
        
        
        cc = stick(v)
        if cc:
            stick_count += 1
            for i in cc:
                edges = [e for e in edges if e not in out_edges[i]]
                if i in q:
                    q.remove(i)
            continue
        
        cc = eight(v)
        if cc:
            eight_count += 1
            for i in cc:
                edges = [e for e in edges if e not in out_edges[i]]
                if i in q:
                    q.remove(i)
            continue
        
    
    answer = [new_vertex, donut_count, stick_count, eight_count]
    return answer











def expect(sol, answer):
    if sol != answer:
        print(f"틀렸음! 정답: {sol}, 하지만 나의 답: {answer}")
    else:
        print("정답!")

# 생성한 정점의 번호, 도넛 모양 그래프의 수, 막대 모양 그래프의 수, 8자 모양 그래프의 수        
for t in tc:
    expect(t[1], solution(t[0]))
    print()
    

정답!

정답!



In [61]:
##########
##########
# 이거 DFS를 재귀적으로 구현하면 프로그래머스에서 런타임 에러 발생한다
# Stack을 활용하여 iteration으로 구현해야 함.
##########
##########

def solution(edges):
    # 현재 그래프에 남아 있는 vertex queue
    vertices = set()
    for src, dst in edges:
        vertices.add(src)
        vertices.add(dst)
    
    
    max_v = max(vertices)
    
    out_edges = dict()
    in_edges = dict()
    
    for v in vertices:
        out_edges[v] = list()
        in_edges[v] = list()
        
    for src, dst in edges:
        out_edges[src].append( (src, dst) )
        in_edges[dst].append( (src, dst) )

    
    new_v = None
    for v in vertices:
        if len(out_edges[v]) >= 2 and len(in_edges[v]) == 0:
            new_v = v
            break
    
    new_edges = out_edges[new_v]
    
    # 그래프에서, 새로 생성된 vertex에서 나오는 edge들을 전부 제거함.
    # edges = [e for e in edges if e not in new_edges]
    # out_edges, in_edges = init_out_and_in_edges()
    
    
    
    visited = [False] * (max_v + 1)

        
    # dfs function call이 너무 많이 쌓여서 메모리를 초과하나?
    def dfs(start, res):
        
        visited[start] = True
        
        stack = [start]
        while stack:
            cur = stack.pop()
            out_edges_num = len(out_edges[cur])
            
            if out_edges_num == 0:
                res['stick'] += 1
                return
            elif out_edges_num == 1:
                src, dst = out_edges[cur][0]
                if not visited[dst]:
                    stack.append(dst)
                    visited[dst] = True
                else:
                    res['donut'] += 1
                    return
            else:
                res['eight'] += 1
                return
    
    result = {
        'donut': 0,
        'stick': 0,
        'eight': 0
    }
    
    for _, dst in new_edges:
        dfs(dst, result)
    
    answer = [new_v, result['donut'], result['stick'], result['eight']]
    
    
    return answer


def expect(sol, answer):
    if sol != answer:
        print(f"틀렸음! 정답: {sol}, 하지만 나의 답: {answer}")
    else:
        print("정답!")

# 생성한 정점의 번호, 도넛 모양 그래프의 수, 막대 모양 그래프의 수, 8자 모양 그래프의 수        
for t in tc:
    expect(t[1], solution(t[0]))
    print()
    

정답!

정답!

